In [23]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
# Read from extracted data
flights_df_raw = pd.read_csv("../data/raw/flights_600k_raw.csv")

In [25]:
# Take copy to measure data loss after clean-up
flights_df = flights_df_raw.copy()

In [26]:
flights_df.columns

Index(['mkt_carrier', 'mkt_carrier_fl_num', 'fl_date', 'branded_code_share',
       'op_unique_carrier', 'op_carrier_fl_num', 'origin_airport_id',
       'origin_city_name', 'dest_airport_id', 'dest_city_name', 'crs_dep_time',
       'dep_time', 'dep_delay', 'taxi_out', 'taxi_in', 'crs_arr_time',
       'arr_time', 'arr_delay', 'cancelled', 'cancellation_code', 'diverted',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime'],
      dtype='object')

#### Group states into four regions


In [27]:
regions = {
    'WA': 'West',
    'OR': 'West',
    'ID': 'West',
    'MT': 'West',
    'WY': 'West',
    'CA': 'West',
    'NV': 'West',
    'UT': 'West',
    'AZ': 'West',
    'CO': 'West',
    'NM': 'West',
    'HI': 'West',
    'AK': 'West',
    
    'ND': 'Midwest',
    'SD': 'Midwest',
    'NE': 'Midwest',
    'KS': 'Midwest',
    'MN': 'Midwest',
    'IA': 'Midwest',
    'MO': 'Midwest',
    'WI': 'Midwest',
    'IL': 'Midwest',
    'IN': 'Midwest',
    'MI': 'Midwest',
    'OH': 'Midwest',

    'OK': 'South',
    'TX': 'South',
    'AR': 'South',
    'LA': 'South',
    'MS': 'South',
    'AL': 'South',
    'TN': 'South',
    'KY': 'South',
    'WV': 'South',
    'DC': 'South',
    'VA': 'South',
    'DE': 'South',
    'MD': 'South',
    'NC': 'South',
    'SC': 'South',
    'GA': 'South',
    'FL': 'South',

    'PA': 'Northeast',
    'NY': 'Northeast',
    'VT': 'Northeast',
    'ME': 'Northeast',
    'NH': 'Northeast',
    'MA': 'Northeast',
    'CT': 'Northeast',
    'RI': 'Northeast',
    'NJ': 'Northeast'
    }

## Create new columns

### 1. Split cities and states. Group states into four regions - 'West', 'Midwest', 'Northeast', 'South'

In [28]:
# Split City and State 
flights_df[['origin_city', 'origin_state']] = flights_df['origin_city_name'].str.split(", ",expand=True,) 
flights_df[['dest_city', 'dest_state']] = flights_df['dest_city_name'].str.split(", ",expand=True,) 

In [29]:
flights_df=flights_df.replace({"origin_state": regions})
flights_df=flights_df.replace({"dest_state": regions})

### 2. Extract day, month, and year

In [30]:
flights_df['day'] = pd.DatetimeIndex(flights_df['fl_date']).day
flights_df['month'] = pd.DatetimeIndex(flights_df['fl_date']).month
flights_df['year'] = pd.DatetimeIndex(flights_df['fl_date']).year

### 3. Four categories of flight time - 'Morning', 'Afternon', 'Evening', and 'Night'

In [31]:
# Converting time into 24 hours
flights_df['crs_arr_hrs'] = (flights_df['crs_arr_time']/100).astype(int)
flights_df['crs_dep_hrs'] = (flights_df['crs_dep_time']/100).astype(int)

In [32]:
# Creating bins & categories for time
bin_time = [0,5,12,16,22]
catg_time = ['Morning','Afternoon','Evening', 'Night']

# Departure hours
dep_hrs_catg = flights_df['crs_dep_hrs']
flights_df['dep_hrs_catg'] = pd.cut(dep_hrs_catg, bin_time, labels=catg_time)

### 4. Day of the week (where Monday is 0)

In [33]:
flights_df['fl_date'][0]

'2019-03-08'

In [34]:
# Conver object to datatime format
flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'])

In [35]:
# Convert datetime column/series to day of the week
flights_df['day_of_week'] = flights_df['fl_date'].dt.weekday

In [36]:
# Replace day numbers with day names
flights_df = flights_df.replace({'day_of_week' : { 
                                                    0 : 'Monday', 
                                                    1 : 'Tuesday', 
                                                    2 : 'Wednesday', 
                                                    3: 'Thursday', 
                                                    4: 'Friday', 
                                                    5: 'Saturday', 
                                                    6: 'Sunday'}})

In [ ]:
# To get the name of the day of the week 
# print(calendar.day_name[my_date.weekday()])

### 5. Day of year

In [37]:
# Convert datetime column/series to year
flights_df['day_of_year'] = flights_df['fl_date'].dt.day_of_year

### 6. Week of year

In [38]:
# Convert datetime column/series to week of year
flights_df['week_of_year'] = flights_df['fl_date'].dt.isocalendar().week

### Check

In [39]:
flights_df.shape

(600000, 46)

In [40]:
flights_df.columns

Index(['mkt_carrier', 'mkt_carrier_fl_num', 'fl_date', 'branded_code_share',
       'op_unique_carrier', 'op_carrier_fl_num', 'origin_airport_id',
       'origin_city_name', 'dest_airport_id', 'dest_city_name', 'crs_dep_time',
       'dep_time', 'dep_delay', 'taxi_out', 'taxi_in', 'crs_arr_time',
       'arr_time', 'arr_delay', 'cancelled', 'cancellation_code', 'diverted',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'origin_city', 'origin_state', 'dest_city',
       'dest_state', 'day', 'month', 'year', 'crs_arr_hrs', 'crs_dep_hrs',
       'dep_hrs_catg', 'day_of_week', 'day_of_year', 'week_of_year'],
      dtype='object')

### Save

In [ ]:
# Save pd as csv
flights_df.to_csv('flights_before_cleaning_v1.csv', index=False)